<h1>logistic regression without applying polynomin</h1>

do regularization with modal 'liblinear'

# <h4>score: private 0.745 - public 0.753</h4>

In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


In [2]:
dataPath = '../../preprocess_data_set/w2v_vec250.csv'


df_train = pd.read_csv(dataPath)
df_train.shape

In [4]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
0,76.089134,111.891899,83.040634,-230.633743,-58.075974,257.050781,33.574692,121.591164,1.534797,-136.893478,...,-150.614395,58.985607,202.579559,-178.152649,236.641144,413.490448,-187.805435,195.260895,-69.905609,0
1,165.381317,148.690323,112.640076,-128.012909,18.291080,83.791595,-81.372841,123.980354,11.166133,17.662382,...,-139.220505,6.865510,88.248108,-196.057449,3.397798,292.449585,-205.278229,81.772591,-107.773155,0
2,35.963501,194.378922,33.754692,-101.244164,-31.473198,58.138603,1.319836,123.404030,-97.246490,-48.950760,...,-70.667305,53.698757,159.106445,-51.326168,105.289841,116.140770,-31.169456,-9.729811,-40.059967,0
3,63.304482,127.738274,-21.535141,-59.371391,-13.245415,45.339760,3.509994,92.514626,-88.460434,-12.191431,...,-70.567566,50.938023,103.309296,-7.114745,86.394791,-0.029950,7.805309,-30.301466,-58.680027,0
4,80.099777,38.637901,27.399906,-279.661102,15.153675,213.821457,55.027672,-46.320877,-6.499963,18.798340,...,-187.330658,55.890907,201.640411,-16.693233,216.730087,100.197075,-216.734711,155.619629,-121.154930,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,71.172447,29.576962,9.814576,-114.468605,10.421734,71.369278,-19.829683,182.287994,-168.829971,-16.698744,...,-98.183441,47.153919,16.384344,7.214799,-16.159613,142.430328,-36.919342,3.927083,-52.101486,1
44894,49.767658,14.141649,24.830746,-39.896648,17.792511,-24.986475,-14.721946,48.350887,12.321180,41.165245,...,-17.996445,15.147109,20.384314,-26.027142,5.624276,43.764351,-58.034969,42.735729,-57.203762,1
44895,21.184694,193.273743,-21.810898,-881.616882,59.446217,742.972473,31.955414,826.063293,-849.959290,-8.474980,...,-1192.223511,209.601578,611.738037,44.896534,436.083008,890.661560,-1118.760498,176.455963,-366.670959,1
44896,-11.720924,-19.239174,26.855413,-27.201494,-16.071684,87.845459,19.739613,129.176987,-113.808846,3.524714,...,-71.951553,18.072672,115.359062,32.927032,64.880394,32.569496,-110.810188,89.154694,-60.245560,1


In [6]:
# randomly shuffle the rows
df_train = df_train.sample(frac=1, random_state=42)

print('Shuffled dataframe:')
df_train

Shuffled dataframe:


,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
3784,5.835878,53.533268,25.216635,-54.122765,-2.916101,48.568226,-18.518307,24.185753,-11.073277,-17.651796,...,-29.318108,14.766320,-3.506311,16.185593,21.054930,5.650350,-10.634307,-8.942313,-6.059765,0
25283,-22.684910,-4.757801,52.808487,-35.903114,-144.107056,-59.875534,-44.122456,77.807480,-70.185432,-31.931232,...,-20.044575,46.117027,92.338531,-70.919563,-28.845901,98.768013,-83.849411,56.284801,-60.075916,1
25292,-15.947605,22.219872,10.545113,-2.965903,-45.021881,3.411819,-20.294031,23.515020,-59.028336,2.427365,...,-18.598761,44.422886,72.973053,-18.207321,-4.930046,37.542679,-26.426054,25.537178,-4.437278,1
640,1.665115,23.308281,-26.644222,-8.566631,-16.841410,31.434685,12.431794,8.679730,-30.553104,-12.451235,...,-48.996002,7.377005,15.708810,-12.438192,41.821358,3.913110,19.218107,16.582424,-17.087742,0
28187,-3.395608,13.553441,-5.400806,-8.400974,-41.418461,20.704168,-17.990662,84.184982,-52.559853,-17.186106,...,-68.018234,41.225601,81.644997,-6.031395,36.422050,53.452858,-24.025400,25.206354,-40.556965,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564,-12.260809,17.981777,13.734811,-1.360468,-47.720970,21.879459,14.012135,29.577761,-43.401501,-48.285572,...,-34.659027,13.330807,23.611446,-34.800346,31.873329,47.460751,-24.528603,18.883064,-8.005676,0
27251,68.259224,42.444344,25.221125,-6.545662,-40.545010,-59.066780,-71.506783,86.965118,-72.739685,-1.060370,...,-33.531174,-24.579193,15.854178,-35.576965,-41.550564,11.299809,24.346041,18.418703,-12.980600,1
4150,23.504272,46.337177,181.560135,-340.232635,-154.115662,185.697983,38.425816,42.398987,50.887287,-52.423264,...,-183.607025,52.148415,35.374458,-24.945963,165.402145,261.748291,-154.710861,102.378387,-67.843636,0
15789,37.916790,0.919980,38.357471,-83.897911,-3.072767,68.915306,26.987322,33.378334,5.194117,-14.573088,...,-35.985695,19.999868,-9.151870,33.420174,21.308874,42.608578,-25.359924,-13.944387,-38.223824,0


In [14]:
df_train = df_train.reset_index()

In [16]:

df_train = df_train.drop('index', axis=1)
df_train

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
0,5.835878,53.533268,25.216635,-54.122765,-2.916101,48.568226,-18.518307,24.185753,-11.073277,-17.651796,...,-29.318108,14.766320,-3.506311,16.185593,21.054930,5.650350,-10.634307,-8.942313,-6.059765,0
1,-22.684910,-4.757801,52.808487,-35.903114,-144.107056,-59.875534,-44.122456,77.807480,-70.185432,-31.931232,...,-20.044575,46.117027,92.338531,-70.919563,-28.845901,98.768013,-83.849411,56.284801,-60.075916,1
2,-15.947605,22.219872,10.545113,-2.965903,-45.021881,3.411819,-20.294031,23.515020,-59.028336,2.427365,...,-18.598761,44.422886,72.973053,-18.207321,-4.930046,37.542679,-26.426054,25.537178,-4.437278,1
3,1.665115,23.308281,-26.644222,-8.566631,-16.841410,31.434685,12.431794,8.679730,-30.553104,-12.451235,...,-48.996002,7.377005,15.708810,-12.438192,41.821358,3.913110,19.218107,16.582424,-17.087742,0
4,-3.395608,13.553441,-5.400806,-8.400974,-41.418461,20.704168,-17.990662,84.184982,-52.559853,-17.186106,...,-68.018234,41.225601,81.644997,-6.031395,36.422050,53.452858,-24.025400,25.206354,-40.556965,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,-12.260809,17.981777,13.734811,-1.360468,-47.720970,21.879459,14.012135,29.577761,-43.401501,-48.285572,...,-34.659027,13.330807,23.611446,-34.800346,31.873329,47.460751,-24.528603,18.883064,-8.005676,0
44894,68.259224,42.444344,25.221125,-6.545662,-40.545010,-59.066780,-71.506783,86.965118,-72.739685,-1.060370,...,-33.531174,-24.579193,15.854178,-35.576965,-41.550564,11.299809,24.346041,18.418703,-12.980600,1
44895,23.504272,46.337177,181.560135,-340.232635,-154.115662,185.697983,38.425816,42.398987,50.887287,-52.423264,...,-183.607025,52.148415,35.374458,-24.945963,165.402145,261.748291,-154.710861,102.378387,-67.843636,0
44896,37.916790,0.919980,38.357471,-83.897911,-3.072767,68.915306,26.987322,33.378334,5.194117,-14.573088,...,-35.985695,19.999868,-9.151870,33.420174,21.308874,42.608578,-25.359924,-13.944387,-38.223824,0


In [18]:
train_y = df_train['target']
train_x = df_train.drop(columns=['target'])


robust_scaler = RobustScaler().fit(train_x.values)
trans_x = robust_scaler.transform(train_x.values)
# trans_test = robust_scaler.transform(test.values)

In [19]:
param_grid = {"C": [.01, .1, 1], "penalty": ('l1','l2')}
logisticClassifier = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter =300, penalty = 'l1')
grid_train = GridSearchCV(logisticClassifier, param_grid, cv = 5)
grid_train.fit(trans_x, train_y)

print('best_score: ', grid_train.best_score_)

best_score:  0.9908904320424966


In [ ]:
pca = PCA(n_components = 200)
pca.fit(train_x.values)
vals_2d = pca.transform(train_x.values)

colors = ['red' if t == 1 else 'blue' for t in train_y]
for i in range(len(vals_2d)):
    plt.scatter(x = vals_2d[i,0], y = vals_2d[i,1], color = colors[i])

In [ ]:
# Create a pipeline with PCA and logistic regression
pc_num = 200

pipe = Pipeline([
    ('pca', PCA(n_components = pc_num)), # choose number of components to keep
    ('clf', LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter = 1000, penalty = 'l1'))
])

pipe.fit(trans_x, train_y)

# Create a scree plot
plt.plot(np.arange(1, pipe['pca'].n_components_+1), pipe['pca'].explained_variance_ratio_, 'bo-')
plt.xlabel('Number of PCs')
plt.ylabel('Proportion of Variance Explained')
plt.title('Scree Plot')
plt.show()

<h3>Real prediction on testing data</h3>

In [24]:
param_grid = {"C": [.01, .1], "penalty": ('l1','l2')}
logisticClassifier = LogisticRegression(solver = 'liblinear', class_weight = 'balanced', max_iter = 300, penalty = 'l1')

# find best parameter with cross validation
grid_test = GridSearchCV(logisticClassifier, param_grid, cv=5)
grid_test.fit(trans_x, train_y)


# predictions = grid_test.predict_proba(trans_test)[:,1]

In [ ]:
print("Average performance of the model: {:.3f}".format(grid_test.best_score_))